Name: Aral Yekta Yarımca
Student Id: 2376093

### Task 1: Naive Formula + Uniform Weighting

In [119]:
import numpy as np
import cv2 as cv
import time

# Window size is assumed as 3x3

def calculate_window(img, row, col):
    result = 0
    result += pow(img[row - 1][col - 1] - img[row, col], 2)
    result += pow(img[row - 1][col] - img[row, col], 2)
    result += pow(img[row - 1][col + 1] - img[row, col], 2)
    result += pow(img[row][col - 1] - img[row, col], 2)
    result += pow(img[row][col] - img[row, col], 2)
    result += pow(img[row][col + 1] - img[row, col], 2)
    result += pow(img[row + 1][col - 1] - img[row, col], 2)
    result += pow(img[row + 1][col] - img[row, col], 2)
    result += pow(img[row + 1][col + 1] - img[row, col], 2)
    return result
    
def harris_corner(img): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols):
            if row == 0 or col == 0 or row == num_rows - 1 or col == num_cols - 1:
                calculated = 0
            else:
                calculated = calculate_window(img, row, col)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = harris_corner(gray)
    # Since non maximal suppression is not required here, I'm getting the largest 10 values for the top 10 corners
    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]
    cv.imwrite(file_name.split('.')[0] + "_output1.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 1.7041454315185547 seconds
agac.png is done in 1.7239112854003906 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 1.6286590099334717 seconds
chessboardrotated.png is done in 1.7122786045074463 seconds
agacrotated.png is done in 1.696251630783081 seconds
labrotated.png is done in 1.6813626289367676 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale.

For the tree image, it took approximately 1.7 seconds for the original and 1.56 seconds for the rotated version. Even though the detected corners reside in the same areas in the image, their exact pixels differ by a small amount.
For the chessboard image, it took approximately 1.56 seconds for the original and 1.59 seconds for the rotated version. The detected corners are exactly the same between the images.
For the lab image, it took approximately 1.7 seconds for the original and 1.62 seconds for the rotated version. 

It can be seen that the detected corners are clustered in similar areas on the image.

### Task2: Naive Formula + Uniform Weighting + Non-Maximum Suppression

In [105]:
import numpy as np
import cv2 as cv
import time

# Window size is assumed as 3x3

def calculate_window(img, row, col):
    result = 0
    result += pow(img[row - 1][col - 1] - img[row, col], 2)
    result += pow(img[row - 1][col] - img[row, col], 2)
    result += pow(img[row - 1][col + 1] - img[row, col], 2)
    result += pow(img[row][col - 1] - img[row, col], 2)
    result += pow(img[row][col] - img[row, col], 2)
    result += pow(img[row][col + 1] - img[row, col], 2)
    result += pow(img[row + 1][col - 1] - img[row, col], 2)
    result += pow(img[row + 1][col] - img[row, col], 2)
    result += pow(img[row + 1][col + 1] - img[row, col], 2)
    return result
    

def harris_corner(img): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols):
            if row == 0 or col == 0 or row == num_rows - 1 or col == num_cols - 1:
                calculated = 0
            else:
                calculated = calculate_window(img, row, col)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = harris_corner(gray)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]
    cv.imwrite(file_name.split('.')[0] + "_output2.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 1.1171038150787354 seconds
agac.png is done in 1.1820759773254395 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 1.2303636074066162 seconds
chessboardrotated.png is done in 1.3092005252838135 seconds
agacrotated.png is done in 1.2481040954589844 seconds
labrotated.png is done in 1.2864491939544678 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale.

For the tree image, it took approximately 1.40 seconds for the original and 1.23 seconds for the rotated version.
For the chessboard image, it took approximately 1.30 seconds for the original and 1.40 seconds for the rotated version.
For the lab image, it took approximately 1.28 seconds for the original and 1.21 seconds for the rotated version.

Aside from the difference between the original images and the rotated images as explained in the previous part, the effect of non-maxima suppression is that it ignores adjacent corners. This means that if adjacent corners are detected, only their maxima is accepted and all others are rejected. This results in less clutter and more precise corners. Also since we used the top 10 corners, removing adjacent corners from this top 10 list allows us to recognize corners from different regions on the images. Even though using non-maxima suppression should normally slow the code down, I believe the code got faster after re-running it multiple times (due to caching).


### Task3: Taylor's Approximation + Uniform Weighting + Non-Maximum Suppression

In [98]:
import numpy as np
import cv2 as cv
import time
import scipy

# Window size is assumed as 3x3

def calculate_window(img, grad_x, grad_y, row, col):
    result = np.zeros((2,2))

    for cur_row in range(row - 1, row + 2):
        for cur_col in range(col - 1, col + 2):
            Ix = grad_x[cur_row][cur_col]
            Iy = grad_y[cur_row][cur_col]
            result += np.array([[pow(Ix, 2), Ix * Iy], [Ix * Iy, pow(Iy, 2)]])
    vec1 = np.array([row, col])
    vec2 = np.array([row, col])
    return (vec1.dot(result)).dot(vec2)

def harris_corner(img, grad_x, grad_y): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols):
            if row < 5 or col < 5 or row >= num_rows - 6 or col >= num_cols - 6:
                calculated = 0
            else:
                calculated = calculate_window(img, grad_x, grad_y, row, col)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst


files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)

    Gx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]]) / 3
    Gy = np.array([[1,1,1],[0,0,0],[-1,-1,-1]]) / 3

    # Get the derivatives
    Ix = scipy.signal.convolve2d(gray, Gx, boundary='symm')
    Iy = scipy.signal.convolve2d(gray, Gy, boundary='symm')

    dst = harris_corner(gray, Ix, Iy)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]

    cv.imwrite(file_name.split('.')[0] + "_output3.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 1.3663561344146729 seconds
agac.png is done in 1.5122716426849365 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 1.648317575454712 seconds
chessboardrotated.png is done in 1.6883893013000488 seconds
agacrotated.png is done in 1.5216262340545654 seconds
labrotated.png is done in 1.6086421012878418 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale. I've also used scipy to convolve matrices.

For the tree image, it took approximately 1.51 seconds for the original and 1.52 seconds for the rotated version.
For the chessboard image, it took approximately 1.36 seconds for the original and 1.68 seconds for the rotated version.
For the lab image, it took approximately 1.64 seconds for the original and 1.60 seconds for the rotated version.

Using Taylor's approximation resulted in an emphasis on the edges instead of the corners. This was the case both on the original images and their rotated versions. I'm guessing the reason for this is the fact that Taylor's approximation misses out some details that are important for corner detection. Furthermore, this approach gave its worse results on the chessboard, failing to detect no corners at all. However, using Taylor's approximation increased the speed of the code.

### Task 4: Smaller Eigenvalue as corner score + Uniform Weighting + Non-Maximum Suppression

In [99]:
import numpy as np
import cv2 as cv
import time
import scipy

# Window size is assumed as 3x3

def calculate_window(img, grad_x, grad_y, row, col):
    result = np.zeros((2,2))

    for cur_row in range(row - 1, row + 2):
        for cur_col in range(col - 1, col + 2):
            Ix = grad_x[cur_row][cur_col]
            Iy = grad_y[cur_row][cur_col]
            result += np.array([[pow(Ix, 2), Ix * Iy], [Ix * Iy, pow(Iy, 2)]])

    return np.linalg.eigvals(result).min()
    

def harris_corner(img, grad_x, grad_y,): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols): # ignore the edges of the image (with a border of 5 pixels)
            if row < 5 or col < 5 or row >= num_rows - 6 or col >= num_cols - 6:
                calculated = 0
            else:
                calculated = calculate_window(img, grad_x, grad_y, row, col)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)

    Gx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]]) / 3
    Gy = np.array([[1,1,1],[0,0,0],[-1,-1,-1]]) / 3

    # Get the derivatives
    Ix = scipy.signal.convolve2d(gray, Gx, boundary='symm')
    Iy = scipy.signal.convolve2d(gray, Gy, boundary='symm')
    
    dst = harris_corner(gray, Ix, Iy)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]

    cv.imwrite(file_name.split('.')[0] + "_output4.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 2.9218599796295166 seconds
agac.png is done in 2.744687557220459 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 2.941112995147705 seconds
chessboardrotated.png is done in 2.7538089752197266 seconds
agacrotated.png is done in 2.74751877784729 seconds
labrotated.png is done in 2.732177972793579 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale. I've also used scipy to convolve matrices.

For the tree image, it took approximately 2.74 seconds for the original and 2.74 seconds for the rotated version.
For the chessboard image, it took approximately 2.92 seconds for the original and 2.75 seconds for the rotated version.
For the lab image, it took approximately 2.94 seconds for the original and 2.73 seconds for the rotated version.

Calculating the eigenvalues definitely slowed the execution speed of the code as it is a costly operation. But the results were much better than the Taylor's approximation approach. The corners were much more precise and most of them were indeed corners. The chessboard outputs were also correct this time. However, there were still some detected points that were not corners.

### Task 5: R function as corner score + Uniform Weighting + Non-Maximum Suppression

In [104]:
import numpy as np
import cv2 as cv
import time
import scipy

# Window size is assumed as 3x3

def calculate_window(img, grad_x, grad_y, row, col, k):
    result = np.zeros((2,2))

    for cur_row in range(row - 1, row + 2):
        for cur_col in range(col - 1, col + 2):
            Ix = grad_x[cur_row][cur_col]
            Iy = grad_y[cur_row][cur_col]
            result += np.array([[pow(Ix, 2), Ix * Iy], [Ix * Iy, pow(Iy, 2)]])

    determinant = np.linalg.det(result)
    trace = np.trace(result)
    return determinant - k * pow(trace, 2)
    

def harris_corner(img, grad_x, grad_y, k): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols): # ignore the edges of the image (with a border of 5 pixels)
            if row < 5 or col < 5 or row >= num_rows - 6 or col >= num_cols - 6:
                calculated = 0
            else:
                calculated = calculate_window(img, grad_x, grad_y, row, col, k)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)

    Gx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]]) / 3
    Gy = np.array([[1,1,1],[0,0,0],[-1,-1,-1]]) / 3

    # Get the derivatives
    Ix = scipy.signal.convolve2d(gray, Gx, boundary='symm')
    Iy = scipy.signal.convolve2d(gray, Gy, boundary='symm')
    
    dst = harris_corner(gray, Ix, Iy, 0.05)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]

    cv.imwrite(file_name.split('.')[0] + "_output5.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 1.7746758460998535 seconds
agac.png is done in 1.983940601348877 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 1.8968935012817383 seconds
chessboardrotated.png is done in 1.8531310558319092 seconds
agacrotated.png is done in 1.9089303016662598 seconds
labrotated.png is done in 1.8825390338897705 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale. I've also used scipy to convolve matrices.

For the tree image, it took approximately 1.98 seconds for the original and 1.90 seconds for the rotated version.
For the chessboard image, it took approximately 1.77 seconds for the original and 1.85 seconds for the rotated version.
For the lab image, it took approximately 1.89 seconds for the original and 1.88 seconds for the rotated version.

Using R function instead of calculating the eigenvalues increased the execution speed of the code. Furthermore, the detected corners also looked similar to the previous (eigenvalue) approach. Even though it still had some wrong detections the overall performance was fine. It also detected some better corners as well (in the lab and the tree images).

### Task 6: R function with fast windowing based on fitering + Uniform Weighting + Non-Maximum Suppression

In [16]:
import numpy as np
import cv2 as cv
import time
import scipy

# Window size is assumed as 3x3

def calculate_window(img, Ix2, Iy2, Ixy, row, col, k):
    Ix2_val = Ix2[row][col]
    Ixy_val = Ixy[row][col]
    Iy2_val = Iy2[row][col]

    return Ix2_val * Iy2_val - pow(Ixy_val, 2) - k * pow((Ix2_val + Iy2_val), 2)
    

def harris_corner(img, Ix2, Iy2, Ixy, k): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols): # ignore the edges of the image (with a border of 5 pixels)
            if row == 0 or col == 0 or row == num_rows - 1 or col == num_cols - 1:
                calculated = 0
            else:
                calculated = calculate_window(img, Ix2, Iy2, Ixy, row, col, k)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)


    Gx2 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    Gy2 = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    gaussian_filter = np.array([[1,1,1], [1,1,1], [1,1,1]]) / 9

    # Get the derivatives
    Ix = scipy.signal.convolve2d(gray, Gx2, boundary='symm')
    Iy = scipy.signal.convolve2d(gray, Gy2, boundary='symm')

    Ix2 = np.square(Ix)
    Iy2 = np.square(Iy)
    Ixy = np.multiply(Ix, Iy)

    gaussIx2 = scipy.signal.convolve2d(Ix2, gaussian_filter)
    gaussIy2 = scipy.signal.convolve2d(Iy2, gaussian_filter)
    gaussIxy = scipy.signal.convolve2d(Ixy, gaussian_filter)

    dst = harris_corner(gray, gaussIx2, gaussIy2, gaussIxy, 0.15)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]

    cv.imwrite(file_name.split('.')[0] + "_output6.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 0.2758331298828125 seconds
agac.png is done in 0.2448110580444336 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 0.25052523612976074 seconds
chessboardrotated.png is done in 0.23191523551940918 seconds
agacrotated.png is done in 0.2172234058380127 seconds
labrotated.png is done in 0.3085145950317383 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale. I've also used scipy to convolve matrices.

For the tree image, it took approximately 0.24 seconds for the original and 0.21 seconds for the rotated version.
For the chessboard image, it took approximately 0.27 seconds for the original and 0.23 seconds for the rotated version.
For the lab image, it took approximately 0.25 seconds for the original and 0.30 seconds for the rotated version.

Using convolution instead of window loops dramatically increased the speed of the code while keeping the corners precise. However, possibly due to a bug, the points were calculated in a neighborhood of 2 pixels around the actual detected corners.

### Task 7: R function with fast windowing based on fitering + Gaussian Weighting + Non-Maximum Suppression

In [17]:
import numpy as np
import cv2 as cv
import time
import scipy

# Window size is assumed as 3x3

def calculate_window(img, Ix2, Iy2, Ixy, row, col, k):
    Ix2_val = Ix2[row][col]
    Ixy_val = Ixy[row][col]
    Iy2_val = Iy2[row][col]

    return Ix2_val * Iy2_val - pow(Ixy_val, 2) - k * pow((Ix2_val + Iy2_val), 2)
    

def harris_corner(img, Ix2, Iy2, Ixy, k): 
    num_rows = len(img)
    num_cols = len(img[0])
    result = []
    for row in range(num_rows):
        calculated_row = []
        for col in range(num_cols): # ignore the edges of the image (with a border of 5 pixels)
            if row == 0 or col == 0 or row == num_rows - 1 or col == num_cols - 1:
                calculated = 0
            else:
                calculated = calculate_window(img, Ix2, Iy2, Ixy, row, col, k)
            calculated_row.append(calculated)
        result.append(np.array(calculated_row))
    return np.array(result)

def nms_window(new_dst, dst, row, col):
    val1 = abs(dst[row - 1][col - 1])
    val2 = abs(dst[row - 1][col])
    val3 = abs(dst[row - 1][col + 1])
    val4 = abs(dst[row][col - 1])
    val5 = abs(dst[row][col + 1])
    val6 = abs(dst[row + 1][col - 1])
    val7 = abs(dst[row + 1][col])
    val8 = abs(dst[row + 1][col + 1])
    center_val = abs(dst[row][col])

    if center_val >= val1 and \
        center_val >= val2 and \
        center_val >= val3 and \
        center_val >= val4 and \
        center_val >= val5 and \
        center_val >= val6 and \
        center_val >= val7 and \
        center_val >= val8:
        new_dst[row][col] = dst[row][col]

def nms(dst_matrix):
    num_rows = len(img)
    num_cols = len(img[0])
    new_dst = np.zeros((num_rows, num_cols))
    for row in range(1, num_rows - 1):
        for col in range(1, num_cols - 1):
            nms_window(new_dst, dst, row, col)
    return new_dst

files = ['chessboard.png', 'agac.png', 'lab.png', 'chessboardrotated.png', 'agacrotated.png', 'labrotated.png']
for file_name in files:
    start = time.time()
    img = cv.imread(file_name)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)


    Gx2 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    Gy2 = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    gaussian_filter = np.array([[1,2,1], [2,4,2], [1,2,1]]) / 16

    # Get the derivatives
    Ix = scipy.signal.convolve2d(gray, Gx2, boundary='symm')
    Iy = scipy.signal.convolve2d(gray, Gy2, boundary='symm')

    Ix2 = np.square(Ix)
    Iy2 = np.square(Iy)
    Ixy = np.multiply(Ix, Iy)

    gaussIx2 = scipy.signal.convolve2d(Ix2, gaussian_filter)
    gaussIy2 = scipy.signal.convolve2d(Iy2, gaussian_filter)
    gaussIxy = scipy.signal.convolve2d(Ixy, gaussian_filter)

    dst = harris_corner(gray, gaussIx2, gaussIy2, gaussIxy, 0.05)

    # Apply non maximal suppression
    dst = nms(dst)

    dst_sorted_indices = np.argsort(dst.reshape(-1))
    sorted_dst = dst.reshape(-1)[dst_sorted_indices]
    max_elements = sorted_dst[-10:] # Get the largest 10 elements
    min_max = max_elements.min()
    img[dst>=min_max]=[0,0,255]

    cv.imwrite(file_name.split('.')[0] + "_output7.png", img)
    end = time.time()
    print(file_name, "is done in", end - start, "seconds")

chessboard.png is done in 0.21144676208496094 seconds
agac.png is done in 0.23963642120361328 seconds


libpng warning: iCCP: known incorrect sRGB profile


lab.png is done in 0.22361278533935547 seconds
chessboardrotated.png is done in 0.22813820838928223 seconds
agacrotated.png is done in 0.26834845542907715 seconds
labrotated.png is done in 0.21793198585510254 seconds


Discussion

I've used Opencv for reading the images and converting them to grayscale. I've also used scipy to convolve matrices.

For the tree image, it took approximately 0.23 seconds for the original and 0.26 seconds for the rotated version.
For the chessboard image, it took approximately 0.21 seconds for the original and 0.22 seconds for the rotated version.
For the lab image, it took approximately 0.22 seconds for the original and 0.21 seconds for the rotated version.

The bug mentioned in the previous task was still present. However, convolving the results with a Gaussian kernel increased the approach's invariance to rotation, meaning rotations did not affect the detected corners as much.